# Delta Calculation Maps

In [1]:
import xarray as xr
import xoak
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import regionmask
import cartopy
import cartopy.crs as ccrs
import glob

In [ ]:
from matplotlib.colors import ListedColormap, BoundaryNorm

def div_colorbar():
    
    div_colors = ['#053061',
                  '#2166ac',
                  '#4393c3',
                  '#92c5de',
                  '#d1e5f0',
                  '#f7f7f7',
                  '#fddbc7',
                  '#f4a582',
                  '#d6604d',
                  '#b2182b',
                  '#67001f'
                  ]
    
    cmap = ListedColormap(div_colors)
    
    return cmap

def div_boundarynorm():
    
    
    cmap = div_colorbar()
    classes = np.array(list(np.linspace(-22.5, 22.5, num=10)))
    norm = BoundaryNorm(classes, ncolors=cmap.N) 
    
    return cmap, norm

cmap, norm = div_boundarynorm()

In [ ]:
from matplotlib import colors
letter = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'm', 'n', 'o', 'p', 'q']

#Plot the data
fig = plt.figure()
plt.rcParams['figure.figsize'] = 13,10
divnorm=colors.TwoSlopeNorm(vmin=-15., vcenter=0., vmax=15)
plt.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.81, 0.15, 0.01, 0.7])

rcp = 8 
i = 1

#iterate through the modes and seasons
for season in seasons:
    for mode in modes:
        plt.tick_params(left = False, right = False , labelleft = False ,
                    labelbottom = False, bottom = False)

        ax = plt.subplot(4, 4, i, projection=ccrs.LambertConformal(central_longitude=-98, central_latitude=35))
        c = ax.pcolormesh(ds_cov_Cellular_eoc4p5_Winter.lon.values, ds_cov_Cellular_eoc4p5_Winter.lat.values, locals()[f'delta_{rcp}p5_hist_{mode}_{season}'],
                          transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm)
        if i <= 4:
            ax.set_title(f'{mode}',fontsize=20)
            
        ax.add_feature(cartopy.feature.STATES, zorder=2)
        ax.add_feature(cartopy.feature.COASTLINE)
        ax.add_feature(cartopy.feature.LAND, facecolor='white')
        ax.add_feature(cartopy.feature.OCEAN, facecolor='lightblue')
        ax.add_feature(cartopy.feature.LAKES, facecolor='lightblue')

        ax.set_extent([-110, -74, 24.5, 50])
        
        ax.annotate(f'{letter[i-1]})', (0.05,0.05),fontsize=16, xycoords='axes fraction')

        
        if i == 1:
            ax.annotate('DJF', xy=(-0.18,0.45), xycoords='axes fraction', fontsize=20, rotation='vertical')
            
        elif i == 5:
            ax.annotate('MAM', xy=(-0.18,0.45), xycoords='axes fraction', fontsize=20, rotation='vertical')
        
        elif i == 9:
            ax.annotate('JJA', xy=(-0.18,0.45), xycoords='axes fraction', fontsize=20, rotation='vertical')

        elif i == 13:
            ax.annotate('SON', xy=(-0.18,0.45), xycoords='axes fraction', fontsize=20, rotation='vertical')
            
        elif i == 4:
            ax.annotate('+', xy=(1.05,0.95), xycoords='axes fraction', fontsize=20)
            
        elif i == 16:
            ax.annotate('_', xy=(1.07,0.17), xycoords='axes fraction', fontsize=20)
            
        i = i +1


        
cbar = plt.colorbar(c, ticks=list(range(-20, 21, 5)), pad=0.01, extend='both', ax=ax, cax=cbar_ax)
cbar.ax.tick_params(labelsize=15)
#cbar.set_label(label='$\Delta$Days', size=10)
plt.show()